# **Дилемма смещения и разброса. Полиномиальные признаки. Регуляризация**

In [6]:
from sklearn import metrics
from sklearn import preprocessing
# всё остальное
from sklearn import linear_model
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
from IPython.display import display
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации
%matplotlib inline
plt.style.use('seaborn') #установка стиля matplotlib

from sklearn.datasets import load_boston 

## **СМЕЩЕНИЕ И РАЗБРОС**

Центральной проблемой всего обучения с учителем (не только линейных моделей) является ***дилемма смещения и разброса модели***. Давайте узнаем, что это такое.

До этого момента мы обучали модели на всех имеющихся данных. С одной стороны, это имеет смысл, ведь мы хотим минимизировать ошибки модели, используя как можно больше данных для обучения.

С другой стороны, из-за такого подхода становится труднее оценивать, насколько хорошо работает модель. Причина этого в том, что, если мы продолжим рассчитывать метрики, используя тренировочные данные, мы можем обнаружить, что при применении модели на незнакомых ей данных она работает довольно плохо.

*Таким образом, модель может детально подстроиться под зависимость в обучающей выборке, но не уловить общей сути.*

Такая проблема называется **переобучением** (overfitting). По сути, такая модель работает намного лучше с обучающими данными, чем с новыми. Она была чрезмерно натренирована на обнаружение уникальных характеристик обучающего набора данных, которые не являются общими закономерностями.

**Недообучение** (underfitting) — проблема, обратная переобучению. Модель из-за своей слабости не уловила никаких закономерностей в данных. В этом случае ошибка будет высокой как для тренировочных данных, так и для данных, не показанных во время обучения.

![](https://lms.skillfactory.ru/assets/courseware/v1/24d0e131092f71d843db3a5fe85cdbf5/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-ml2-5_1.png)

* На первом рисунке изображена простая модель линейной регрессии, не способная уловить сложную зависимость в данных.
* На втором рисунке изображена оптимальная модель, которая хорошо описывает зависимость и при этом не имеет переобучения (полином четвёртой степени).
* На последнем рисунке изображен полином 27-й степени, который подстроился под каждую точку в тренировочном наборе, но не смог уловить общие закономерности.
***
С теоретической точки зрения недообучение и переобучение характеризуются понятиями **смещения** и **разброса** модели.
***
**Смещение (bias)** — это математическое ожидание разности между истинным ответом и ответом, выданным моделью. То есть это **ожидаемая ошибка модели**.

![](data\f49.png)

В зарубежной литературе математическое ожидание часто обозначается как ***E***:

![](data\f50.png)

Чем больше смещение, тем слабее модель. Если модель слабая, она не в состоянии выучить закономерность. Таким образом, налицо недообучение модели.
***
**Разброс (variance)** — это вариативность ошибки, то, насколько ошибка будет отличаться, если обучать модель на разных наборах данных. Математически это дисперсия (разброс) ответов модели.

![](data\f51.png)

Примечание. В зарубежной литературе дисперсия часто обозначается как ***Var***:

![](data\f52.png)

Чем больше разброс, тем больше ошибка будет колебаться на разных наборах данных. Наличие высокого разброса и есть свидетельство переобучения: модель подстроилась под конкретный набор данных и даёт высокий разброс ответов на разных данных.
***
Теоретически на составляющие смещения и разброса модели можно разложить любую функцию потерь. Например, разложение квадратичной ошибки (её математическое ожидание) будет выглядеть следующим образом:

![](data\f53.png)

Математическое ожидание — это среднее значение во всей генеральной совокупности (на бесконечной выборке), а не на конкретных значениях. Это исключительно теоретическая величина.

* ***σ*** — неустранимая ошибка, вызванная случайностью.
* ***bias(y)^2*** — смещение модели (в квадрате).
* ***variance(y)*** — разброс модели.

**О чём нам говорит эта теоретическая формула?**

Ошибка модели складывается из смещения модели (в квадрате) и её разброса, а также случайной ошибки. 

Если с последним слагаемым  мы ничего не сможем сделать, то вот на первые два (bias и variance) мы можем как-то повлиять. В идеале мы должны свести их к 0. Однако уменьшение одного слагаемого повлечёт увеличение другого. На практике часто приходится балансировать между смещёнными и нестабильными оценками.
Дилемма смещения-дисперсии является центральной проблемой в обучении с учителем. В идеале мы хотим построить модель, которая точно описывает зависимости в тренировочных данных и хорошо работает на неизвестных данных. К сожалению, обычно это невозможно сделать одновременно.

* **Усложняя** модель, мы пытаемся **уменьшить смещение (bias)**, однако появляется риск получить переобучение, то есть мы **повышаем разброс (variance)**. 
* С другой стороны, **снизить разброс (variance)** позволяют более **простые модели**, не склонные к переобучению, но есть риск, что простая модель не уловит зависимостей и окажется недообученной, то есть мы **повышаем смещение (bias)**.

Если провести аналогию, что модель — это игрок в дартс, то самый лучший игрок будет иметь небольшое смещение и разброс: его дротики ложатся кучно «в яблочко». Если у игрока большое смещение, то дротики сгруппированы около другой точки, а если большой разброс — дротики разлетаются по всей мишени.

![](https://lms.skillfactory.ru/assets/courseware/v1/c9e31e930c613a02ef6abd7c363158d6/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-ml2-5_2.png)
***
Теперь, когда мы знаем о теоретических основах проблемы переобучения и недообучения, что мы можем сделать, чтобы лучше судить о способности модели к обобщению на практике? Как диагностировать высокие bias и variance?

Типичным решением является разделение данных на две части: **обучающий** и **тестовый** наборы. На тренировочном наборе данных мы будем обучать модель, подбирая параметры. Тестовый набор данных, который модель не видела при обучении, мы будем использовать для оценки истинного качества моделирования. Схематично можно представить это следующим образом:

![](https://lms.skillfactory.ru/assets/courseware/v1/6b3371e3f1d290f0c216ac1e826005ff/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-ml2-5_3.png)



Давайте посмотрим, как это работает на практике. Работать будем с уже знакомыми нам данными — данными о домах в Бостоне.

In [2]:
from sklearn.datasets import load_boston 

boston = load_boston()
#создаём DataFrame из загруженных numpy-матриц
boston_data = pd.DataFrame(
    data=boston.data, #данные
    columns=boston.feature_names #наименования столбцов
)
#добавляем в таблицу столбец с целевой переменной
boston_data['MEDV'] = boston.target
 
#Составляем список факторов (исключили целевой столбец)
features = boston_data.drop('MEDV', axis=1).columns
#Составляем матрицу наблюдений X и вектор ответов y
X = boston_data[features]
y = boston_data['MEDV']

c:\Users\local_admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets inclu

В **sklearn** для разделения выборки на тренировочную и тестовую есть функция [**train_test_split()**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) из модуля **model_selection**. Данная функция принимает следующие аргументы:

* ***X*** и ***y*** — таблица с примерами и ответами к ним.
* ***random_state*** — число, на основе которого генерируются случайные числа. Тренировочная и тестовая выборка генерируются случайно. Чтобы эксперимент был воспроизводимым, необходимо установить этот параметр в конкретное значение.
* ***test_size*** — доля тестовой выборки. Параметр определяет, в каких пропорциях будет разделена выборка. Стандартные значения: 70/30, 80/20.

Функция возвращает четыре объекта в следующем порядке: тренировочные примеры, тестовые примеры, тренировочные ответы и тестовые ответы. 

Итак, давайте разделим нашу выборку на тренировочную и тестовую в соотношении 70/30:

In [3]:
from sklearn.model_selection import train_test_split
#Разделяем выборку на тренировочную и тестовую в соотношении 70/30
#Устанавливаем random_state для воспроизводимости результатов 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=40)
#Выводим результирующие размеры таблиц
print('Train:', X_train.shape, y_train.shape)
print('Test:', X_test.shape, y_test.shape)

Train: (354, 13) (354,)
Test: (152, 13) (152,)


После разделения в тренировочной выборке оказались 354 наблюдения, а в тестовой — 152.

Затем обучим линейную регрессию (с помощью МНК) на тренировочных данных и рассчитаем R^2 для тренировочных и тестовых данных:

In [4]:
#Создаём объект класса LinearRegression
lr_model = linear_model.LinearRegression()
#Обучаем модель по МНК
lr_model.fit(X_train, y_train)
 
#Делаем предсказание для тренировочной выборки
y_train_predict = lr_model.predict(X_train)
#Делаем предсказание для тестовой выборки
y_test_predict = lr_model.predict(X_test)
 
print("Train R^2: {:.3f}".format(metrics.r2_score(y_train, y_train_predict)))
print("Test R^2: {:.3f}".format(metrics.r2_score(y_test, y_test_predict)))

Train R^2: 0.743
Test R^2: 0.722


Итак, ***R^2 = 0.743*** на тренировочной выборке и ***R^2 = 0.722*** на тестовой выборке. То есть показатели довольно близки друг к другу (низкий разброс ответов модели для разных выборок).

Это одно из свидетельств отсутствия переобучения. Это не удивительно, ведь линейная регрессия, построенная на 13 факторах, является довольно простой моделью: всего лишь 14 параметров, что очень мало по меркам машинного обучения. Риск переобучения возрастает с количеством факторов, которые участвуют в обучении модели.

Но что насчёт смещения? Самый простой способ оценить смещение и недообученность модели — посмотреть на значение метрики и интуитивно оценить её.

Может, наша модель слишком слабая? ***R^2 = 0.722*** — не слишком уж высокий показатель (напомним, максимум — 1). Возможно, стоит попробовать обучить более сложную модель. Например, можно построить модель полиномиальной регрессии.

Примечание. Существуют более совершенные визуальные способы оценить наличие смещения и разброса, такие как кривая обучения и кросс-валидация. О них мы ещё поговорим далее в курсе.

## **ПОЛИНОМИАЛЬНЫЕ ПРИЗНАКИ**

**Полиномиальная регрессия (Polynomial Regression)** — это более сложная модель, чем линейная регрессия. Вместо уравнения прямой используется уравнение полинома (многочлена). Степень полинома может быть сколь угодно большой: чем больше степень, тем сложнее модель.

В простом двумерном случае, когда мы рассматриваем зависимость целевого признака от одного фактора, полиномом второй степени будет уравнение параболы:

![](data\f55.png)

Геометрически полином в двумерном пространстве — это некоторая кривая, которая пытается описать зависимость в данных. Выглядит это следующим образом:

![](https://lms.skillfactory.ru/assets/courseware/v1/01a58ba6150976bb3c408323a9b5fbcf/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-ml2-5_5.png)

Когда факторов больше одного, например два, то, помимо возведения фактора в квадрат, появляются ещё и комбинации из ***x1*** и ***x2***:

![](data\f56.png)

Такая модель будет описывать сложную поверхность в трёхмерном пространстве, которая проставлена на рисунке:

![](https://lms.skillfactory.ru/assets/courseware/v1/10616697357bc2252dd39b8fa419ffc0/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-ml2-5_6.png)

Заметьте, как быстро растёт количество коэффициентов, а с ним и сложность модели. А ведь это только два фактора — ***x1*** и ***x2***. Мы не будем приводить уравнение для общего случая, как делали это с линейной регрессией, так как оно будет содержать слишком много слагаемых.

Например, рассматривая построенный ранее график зависимости медианной цены домов от процента низкостатусного населения, вы могли заметить, что между данными показателями существует не линейная, а скорее полиномиальная связь.

![](https://lms.skillfactory.ru/assets/courseware/v1/4b02bf0809fb707c3950ce0aa275368e/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-ml2-5_7.png)

Заметим, что степени ***x*** можно тоже считать своего рода искусственными признаками в данных. Они называются полиномиальными признаками.

Поэтому полиномиальная регрессия — это та же линейная регрессия, просто с новыми признаками. Полиномиальные признаки — один из самых распространённых методов FeatureEngineering.

Пример:

![](https://lms.skillfactory.ru/assets/courseware/v1/c9664ed6fac637130e49aa29ebfb67e9/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-ml2-5_8.png)

Благодаря степенным слагаемым модель становится сложнее и начинает улавливать более сложные зависимости и выдавать меньшее смещение. Но, как вы понимаете, резко повышается риск переобучения модели — увеличивается разброс предсказаний на разных данных из-за количества факторов.
***
Давайте проследим за этим.

Построить полиномиальную регрессию в sklearn очень просто. Для начала необходимо создать полиномиальные признаки с помощью объекта класса [**PolynomialFeatures**](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) из модуля **preprocessing**. Это преобразователь, который позволит сгенерировать полиномиальные признаки любой степени и добавить их в таблицу. У него есть два важных параметра:

* **degree** — степень полинома. По умолчанию используется степень 2.
* **include_bias** — включать ли в результирующую таблицу столбец из единиц (x в степени 0). По умолчанию стоит True, но лучше выставить его в значение **False**, так как столбец из единиц и так добавляется в методе наименьших квадратов.

*Примечание. Как правило, дата-сайентисты останавливаются на полиноме второй (максимум третьей) степени. Чем выше степень полинома, тем больше слагаемых, а значит, тем больше признаков и тем сложнее становится модель.*

Для того чтобы подогнать генератор и рассчитать количество комбинаций степеней, мы используем метод **fit()**, а чтобы сгенерировать новую таблицу признаков, в которую будут включены полиномиальные признаки, используется метод **transform()**, в который нужно передать выборки:


In [7]:
#Создаём генератор полиномиальных признаков
poly = preprocessing.PolynomialFeatures(degree=2, include_bias=False)
poly.fit(X_train)
#Генерируем полиномиальные признаки для тренировочной выборки
X_train_poly = poly.transform(X_train)
#Генерируем полиномиальные признаки для тестовой выборки
X_test_poly = poly.transform(X_test)
#Выводим результирующие размерности таблиц
print(X_train_poly.shape)
print(X_test_poly.shape)

(354, 104)
(152, 104)


In [8]:
# В результате мы получили два numpy-массива:
print(type(X_train_poly))
print(type(X_test_poly))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [9]:
# Теперь попробуем скормить наши данные модели линейной регрессии, чтобы найти коэффициенты полинома по МНК-алгоритму:

#Создаём объект класса LinearRegression
lr_model_poly = linear_model.LinearRegression()
#Обучаем модель по МНК
lr_model_poly.fit(X_train_poly, y_train)
#Делаем предсказание для тренировочной выборки
y_train_predict_poly = lr_model_poly.predict(X_train_poly)
#Делаем предсказание для тестовой выборки
y_test_predict_poly = lr_model_poly.predict(X_test_poly)
 
#Рассчитываем коэффициент детерминации для двух выборок
print("Train R^2: {:.3f}".format(metrics.r2_score(y_train, y_train_predict_poly)))
print("Test R^2: {:.3f}".format(metrics.r2_score(y_test, y_test_predict_poly)))

Train R^2: 0.929
Test R^2: 0.268


Потрясающе! На тренировочной выборке коэффициент детерминации **R^2 = 0.929**, то есть наша модель описывает почти 93 % зависимости в данных.

Смотрим на показатели тестовой выборки и сразу «спускаемся с небес на землю»: **R^2 = 0.268**. Метрика значительно ниже, чем на тренировочном наборе. Это и есть переобучение модели. Из-за своей сложности (количества факторов) модель полностью адаптировалась под тренировочные данные, но взамен получила высокий разброс в показателях на данных, которые она не видела ранее. 

Такая модель никому не нужна, так как она не отражает действительности.

Однако не стоит расстраиваться — есть один замечательный метод, который сможет спасти нашу модель от переобучения, и это **регуляризация**.

In [ ]:
#Создаём генератор полиномиальных признаков
poly = preprocessing.PolynomialFeatures(degree=2, include_bias=False)
poly.fit(pd.dataframe())
#Генерируем полиномиальные признаки для тренировочной выборки
X_train_poly = poly.transform(X_train)
#Генерируем полиномиальные признаки для тестовой выборки
X_test_poly = poly.transform(X_test)
#Выводим результирующие размерности таблиц
print(X_train_poly.shape)
print(X_test_poly.shape)